# Free-for-all practice file, not included in final submission

In [1]:
import math

from bloqade import qasm2
from kirin.dialects import ilist

In [7]:
def ghz_linear(n: int):
    n_qubits = int(2**n)

    @qasm2.extended
    def ghz_linear_program():

        qreg = qasm2.qreg(n_qubits)
        # Apply a Hadamard on the first qubit
        qasm2.h(qreg[0])
        # Create a cascading sequence of CX gates
        # necessary for quantum computers that
        # only have nearest-neighbor connectivity between qubits
        for i in range(1, n_qubits):
            qasm2.cx(qreg[i - 1], qreg[i])

    return ghz_linear_program

In [4]:
from bloqade.qasm2.emit import QASM2 # the QASM2 target
from bloqade.qasm2.parse import pprint # the QASM2 pretty printer

# Ensure the ghz_linear function emits operations compatible with QASM2
from bloqade.qasm2.dialects import core

# Register the required dialects
target = QASM2(dialects=[core])

# Emit the AST using the updated target
ast = target.emit(ghz_linear(2))
pprint(ast)

TypeError: QASM2.__init__() got an unexpected keyword argument 'dialects'

The webpage for source code: https://github.com/QuEraComputing/bloqade-pyqrack/blob/main/src/bloqade/pyqrack/qasm2/parallel.py



In [5]:
from typing import Any

from bloqade.qasm2.dialects import parallel


In [9]:
import kirin.interp



@qasm2.extended
def global_H():
    qreg = qasm2.qreg(3)
    
    qasm2.parallel.u(
        qreg,          # Pass the list of qubits
        theta=math.pi/2, # Specify the theta angle
        phi=0,           # Specify the phi angle
        lam=math.pi      # Specify the lambda angle
    )
    qasm2.zz(qreg[0])
    qasm2.h(qreg[1])
    qasm2.zz(qreg[1])
    qasm2.h(qreg[2])
    return global_H


target = QASM2()
ast = target.emit(ghz_linear(2))
pprint(ast)


ValueError: dialect py.constant is not supported by DialectGroup([lowering.call, qasm2.expr, scf, func, qasm2.uop, qasm2.core, qasm2.indexing, lowering.func])